# Multi-Analyst Text Analysis Pipeline

This notebook demonstrates the full pipeline for analyzing text through multiple specialist lenses (rhetorician, syntactician, lexicologist, etc.) and synthesizing their observations.

## Installations and Preparations
First, external modules are installed and ensured to be in working order.

In [1]:
# Optional: Install requirements if running in a fresh kernel
# Uncomment if needed:
!pip install -r requirements.txt

# Or install individual packages:
# !pip install litellm pydantic jinja2


[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

Providers
* openai
* openai_like
* bytez
* xai
* custom_openai
* text-completion-openai
* cohere
* cohere_chat
* clarifai
* anthropic
* anthropic_text
* replicate
* huggingface
* together_ai
* datarobot
* openrouter
* cometapi
* vertex_ai
* vertex_ai_beta
* gemini
* ai21
* baseten
* azure
* azure_text
* azure_ai
* sagemaker
* sagemaker_chat
* bedrock
* vllm
* nlp_cloud
* petals
* oobabooga
* ollama
* ollama_chat
* deepinfra
* perplexity
* mistral
* groq
* nvidia_nim
* cerebras
* baseten
* ai21_chat
* volcengine
* codestral
* text-completion-codestral
* deepseek
* sambanova
* maritalk
* cloudflare
* fireworks_ai
* friendliai
* watsonx
* watsonx_text
* triton
* predibase
* databricks
* empower
* github
* custom
* litellm_proxy
* hosted_vllm
* llamafile
* lm_studio
* galadriel
* gradient_ai
* github_copilot
* novita
* meta_llama
* featherless_ai
* nscale
* nebius
* dashscope
* moonshot
* v0
* heroku
* oci
* morph
* lambda_ai
* vercel_ai_gateway
* wandb
* ovhcloud
* lemonade


## Initialize Base Objects
Set up connections to a Large Language Model provider via `litellm` model router. Also, setup up tools to retrieve text data to be part of the context window, that is, instruction prompts and texts to analyze. A basic result storage is also initialized.

The LLM to use is set by the `model_string`, which is constructed as `<provider>/<model>`, the providers defined by the `litellm` package, see in particular `litellm.LITELLM_CHAT_PROVIDERS`. The API key to the provider should be stored in an environment variable with name defined in `model_provider_api_key_env_var`. Do **not** store the API key as a string variable directly in the notebook.

In [3]:
model_string = 'mistral/mistral-large-2411'
model_provider_api_key_env_var = 'MISTRAL_API_KEY'

In [4]:
import os
from pathlib import Path
from belletrist import LLM, LLMConfig, PromptMaker, DataSampler, ResultStore

llm = LLM(LLMConfig(
    model=model_string,
    api_key=os.environ.get(model_provider_api_key_env_var)
))
prompt_maker = PromptMaker()
sampler = DataSampler()
store = ResultStore(Path(f"{os.getcwd()}/belletrist_storage.db"))

In case a clean run is done, the old contents of the database are discarded with a result store reset. Do not run the rest if content should be preserved.

In [5]:
store.reset()

## Generate and Store Text Samples to be Analyzed

A random text sample is taken from the data corpus and stored with full provenance (which file, which paragraphs). Each sample is an instance of `TextSegment`.

The sample size is set by the variable `n_sample` and each sample comprises `m_paragraphs_per_sample` number of consecutive paragraphs.

If non-random text samples are preferred, use the `get_paragraph_chunk` method of the `DataSampler` instance.

In [6]:
text_sample = sampler.sample_segment(p_length=4)
print(f'Text source: {text_sample.file_path}')
print(f'Paragraph range: {text_sample.paragraph_start} - {text_sample.paragraph_end}')
print(f'\n{text_sample.text}')

Text source: /Users/andersohrn/PycharmProjects/russell_writes/data/russell/mysticism_and_logic_and_other_essays.txt
Paragraph range: 365 - 369

Consider for example the infinite divisibility of matter. In looking
at a given thing and approaching it, one sense-datum will become
several, and each of these will again divide. Thus _one_ appearance
may represent _many_ things, and to this process there seems no end.
Hence in the limit, when we approach indefinitely near to the thing
there will be an indefinite number of units of matter corresponding to
what, at a finite distance, is only one appearance. This is how
infinite divisibility arises.

The whole causal efficacy of a thing resides in its matter. This is in
some sense an empirical fact, but it would be hard to state it
precisely, because "causal efficacy" is difficult to define.

What can be known empirically about the matter of a thing is only
approximate, because we cannot get to know the appearances of the
thing from very small d

In [7]:
n_sample = 1
m_paragraphs_per_sample = 10

for _ in range(n_sample):
    sample_id = f'sample_{len(store.list_samples()) + 1:03d}'
    segment = sampler.sample_segment(p_length=m_paragraphs_per_sample)
    store.save_segment(sample_id, segment)


In [8]:
print('Sample keys:\n============')
store.list_samples()

Sample keys:


['sample_001']

## Construct the Analyst Agents and Analyze

Send the text samples through each specialist analyst. Each produces an independent analysis from their domain expertise.

**Prompt structure for caching optimization:**
1. Preamble instruction (static)
2. Analyst-specific template (static per analyst)
3. Text to analyze (dynamic)

Note that the execution of this can take time since it involves invoking LLMs. These are however independent analyses, and can therefore in principle be run in parallel, though the implementation below does not utilize that fact.

In [9]:
from belletrist.models import (
    PreambleInstructionConfig,
    PreambleTextConfig,
    RhetoricianConfig,
    SyntacticianConfig,
    LexicologistConfig,
    InformationArchitectConfig,
    EfficiencyAuditorConfig,
    CrossPerspectiveIntegratorConfig,
)
ANALYSTS = ["rhetorician", "syntactician", "lexicologist", "information_architect", "efficiency_auditor"]
ANALYST_CONFIGS = {
    "rhetorician": RhetoricianConfig,
    "syntactician": SyntacticianConfig,
    "lexicologist": LexicologistConfig,
    "information_architect": InformationArchitectConfig,
    "efficiency_auditor": EfficiencyAuditorConfig,
}

def build_analyst_prompt(preamble_instruction: str, analyst_prompt: str, preamble_text: str) -> str:
    """
    Helper function to construct the full prompt for an analyst.
    
    """
    return f"{preamble_instruction}\n\n{analyst_prompt}\n\n{preamble_text}"

In [10]:
# Get all samples from the store
all_samples = store.list_samples()
print(f"Processing {len(all_samples)} samples with {len(ANALYSTS)} analysts each\n")

# Outer loop: iterate over each text sample
for sample_id in all_samples:
    print(f"Sample: {sample_id}")
    
    # Get the sample text
    sample = store.get_sample(sample_id)
    text = sample['text']
    
    # Build shared prompt components (reused across all analysts for this sample)
    preamble_instruction = prompt_maker.render(PreambleInstructionConfig())
    preamble_text = prompt_maker.render(PreambleTextConfig(text_to_analyze=text))
    
    # Inner loop: run each analyst on this sample
    for analyst_name in ANALYSTS:
        print(f"  Running {analyst_name}...", end=" ")
        
        # Get analyst-specific prompt using the config class
        analyst_config = ANALYST_CONFIGS[analyst_name]()
        analyst_prompt = prompt_maker.render(analyst_config)
        
        # Build full prompt using helper function
        full_prompt = build_analyst_prompt(preamble_instruction, analyst_prompt, preamble_text)
        
        # Run analysis
        response = llm.complete(full_prompt)
        store.save_analysis(sample_id, analyst_name, response.content, response.model)
        
        print(f"✓ ({len(response.content)} chars)")
    
    print()  # Blank line between samples

print(f"All analyses complete for {len(all_samples)} samples")

Processing 1 samples with 5 analysts each

Sample: sample_001
  Running rhetorician... ✓ (3612 chars)
  Running syntactician... ✓ (8566 chars)
  Running lexicologist... ✓ (5885 chars)
  Running information_architect... ✓ (9118 chars)
  Running efficiency_auditor... ✓ (5032 chars)

All analyses complete for 1 samples


Then verify that analysis was run as expected and yielded analysis.

In [11]:
sample_id = 'sample_001'
is_complete = store.is_complete(sample_id, ANALYSTS)
print(f"Analysis complete: {is_complete}")

# Retrieve sample and all analyses (both are now dicts)
sample, analyses = store.get_sample_with_analyses(sample_id)

print(f"\nSample: {sample['sample_id']}")
print(f"Source: File {sample['file_index']}, paragraphs {sample['paragraph_start']}-{sample['paragraph_end']}")
print(f"Analyses available: {list(analyses.keys())}")

# Examine one analysis
print(f"\n--- Rhetorician Output (first 500 chars) ---")
print(analyses.get("rhetorician", "Not found")[:500])

Analysis complete: True

Sample: sample_001
Source: File 0, paragraphs 9-19
Analyses available: ['efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician']

--- Rhetorician Output (first 500 chars) ---
Based on the provided text, here's a rhetorical analysis focusing on rhetorical strategy and stance:

1. **WRITER'S POSITION**

   - **Persona**: The writer emerges as an authority in the field, with a formal, academic tone. The use of specialized terms (_à priori_, _apodeictic_, _hypothetical_, _categorical_, etc.) and references to prominent figures (Kant, Bradley) and scholarly works (_Prolegomena_, _Pure Reason_) establishes their credentials.
   - **Relationship to subject matter**: The wri


## Pattern Recognition (Cross-Perspective Integration)

Synthesize all analyst perspectives to identify interactions, tensions, and load-bearing features. This is a per-text-cross-analyst transformation where a unit of writing patterns for each text sample is generated.

In [12]:
samples_to_analyze = store.list_samples()[0:1]
print(samples_to_analyze)

['sample_001']


In [13]:
def build_pattern_prompt_from_(text: str, analyses: dict):
    sample, analyses = store.get_sample_with_analyses(sample_id)

    analyst_info = {}
    for analyst_name in ANALYSTS:
        config_class = ANALYST_CONFIGS[analyst_name]
        analyst_info[analyst_name] = {
            'analysis': analyses[analyst_name],
            'analyst_descr_short': config_class.description()
        }

    pattern_config = CrossPerspectiveIntegratorConfig(
        original_text=sample['text'],
        analysts=analyst_info
    )
    return prompt_maker.render(pattern_config)

In [14]:
for sample_id in samples_to_analyze:
    print(f"Running pattern recognizer for {sample_id}...", end=" ")
    
    sample, analyses = store.get_sample_with_analyses(sample_id)
    pattern_prompt = build_pattern_prompt_from_(text=sample['text'], analyses=analyses)

    pattern_response = llm.complete(pattern_prompt)
    
    # Store pattern recognition result in result store
    store.save_analysis(
        sample_id, 
        CrossPerspectiveIntegratorConfig.analyst_name(), 
        pattern_response.content, 
        pattern_response.model
    )
    
    print(f"✓ ({len(pattern_response.content)} chars)")

Running pattern recognizer for sample_001... ✓ (11791 chars)


In [15]:
sample_id = 'sample_001'
sample, analyses = store.get_sample_with_analyses(sample_id)
print(analyses.keys())

print(f"\n--- Pattern Analyst Output (first 2000 chars) ---")
print(analyses.get("pattern_recognizer", "Not found")[:2000])

dict_keys(['cross_perspective_integrator', 'efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician'])

--- Pattern Analyst Output (first 2000 chars) ---
Not found


## Stage 2: Cross-Text Synthesis

Now we synthesize patterns across multiple text analyses. This stage takes all the pattern recognition outputs (cross-perspective integrations from Stage 1) and identifies:
- Recurring patterns across texts
- Context-dependent variations
- Hierarchies of importance
- Generalizable principles

This produces a single synthesis document that IS stored in the ResultStore with auto-generated ID and full provenance tracking.

In [ ]:
from belletrist.models import CrossTextSynthesizerConfig

# Get all samples that have pattern recognition results
all_samples = store.list_samples()
pattern_analyst = CrossPerspectiveIntegratorConfig.analyst_name()

# Retrieve all pattern recognition analyses
integrated_analyses = {}
for sample_id in all_samples:
    pattern_analysis = store.get_analysis(sample_id, pattern_analyst)
    if pattern_analysis:
        integrated_analyses[sample_id] = pattern_analysis
    else:
        print(f"⚠ Sample {sample_id} missing pattern recognition analysis")

print(f"Found {len(integrated_analyses)} pattern recognition analyses")
print(f"Sample IDs: {list(integrated_analyses.keys())}")

# Only proceed if we have at least 2 (required for cross-text synthesis)
if len(integrated_analyses) >= 2:
    # Build cross-text synthesis config and prompt
    cross_text_config = CrossTextSynthesizerConfig(
        integrated_analyses=integrated_analyses
    )
    cross_text_prompt = prompt_maker.render(cross_text_config)
    
    # Run cross-text synthesis
    print("\nRunning cross-text synthesis...", end=" ")
    cross_text_response = llm.complete(cross_text_prompt)
    print(f"✓ ({len(cross_text_response.content)} chars)")
    
    # Save to ResultStore with auto-generated ID and full provenance
    sample_contributions = [(sid, pattern_analyst) for sid in integrated_analyses.keys()]
    cross_text_id = store.save_synthesis(
        synthesis_type=cross_text_config.synthesis_type(),
        output=cross_text_response.content,
        model=cross_text_response.model,
        sample_contributions=sample_contributions,
        config=cross_text_config
    )
    
    print(f"Saved as: {cross_text_id}")
    
    # Display first part
    print("\n--- Cross-Text Synthesis (first 1000 chars) ---")
    print(cross_text_response.content[:1000])
else:
    print(f"\n⚠ Need at least 2 pattern recognition analyses for cross-text synthesis. Got {len(integrated_analyses)}.")

## Stage 3: Synthesizer of Principles

The final stage converts the descriptive cross-text synthesis into actionable prescriptive writing principles. This generates a style guide that can be used to instruct an LLM to write in a similar style.

In [ ]:
from belletrist.models import SynthesizerOfPrinciplesConfig

# Get the latest cross-text synthesis
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
if cross_text_syntheses:
    # Use the most recent cross-text synthesis
    latest_cross_text_id = cross_text_syntheses[-1]
    cross_text_synthesis = store.get_synthesis(latest_cross_text_id)
    
    print(f"Using cross-text synthesis: {latest_cross_text_id}")
    
    # Build principles guide config and prompt
    principles_config = SynthesizerOfPrinciplesConfig(
        synthesis_document=cross_text_synthesis['output']
    )
    principles_prompt = prompt_maker.render(principles_config)
    
    # Run principles synthesis
    print("Running principles synthesizer...", end=" ")
    principles_response = llm.complete(principles_prompt)
    print(f"✓ ({len(principles_response.content)} chars)")
    
    # Save to ResultStore with parent linkage (inherits provenance)
    principles_id = store.save_synthesis(
        synthesis_type=principles_config.synthesis_type(),
        output=principles_response.content,
        model=principles_response.model,
        sample_contributions=[],  # Inherited from parent
        config=principles_config,
        parent_synthesis_id=latest_cross_text_id
    )
    
    print(f"Saved as: {principles_id}")
    
    # Display first part
    print("\n--- Principles Guide (first 1000 chars) ---")
    print(principles_response.content[:1000])
else:
    print("⚠ No cross-text synthesis found. Run Stage 2 first.")

## Querying Synthesis Metadata

The ResultStore tracks full provenance for all syntheses. Query metadata to understand what samples, analysts, and models contributed to each synthesis.

In [ ]:
# List all syntheses
print("All Syntheses")
print("=" * 50)
for synth_type in ['cross_text_synthesis', 'principles_guide']:
    syntheses = store.list_syntheses(synth_type)
    print(f"\n{synth_type}: {len(syntheses)} found")
    for synth_id in syntheses:
        print(f"  - {synth_id}")

# Get detailed metadata for a synthesis
if store.list_syntheses():
    print("\n\nDetailed Metadata Example")
    print("=" * 50)
    
    # Get first principles guide (or first cross-text if none)
    principles = store.list_syntheses('principles_guide')
    if principles:
        synth_id = principles[0]
    else:
        synth_id = store.list_syntheses()[0]
    
    synth_with_meta = store.get_synthesis_with_metadata(synth_id)
    
    print(f"\nSynthesis ID: {synth_with_meta['synthesis_id']}")
    print(f"Type: {synth_with_meta['type']}")
    print(f"Model: {synth_with_meta['model']}")
    print(f"Created: {synth_with_meta['created_at']}")
    print(f"Parent: {synth_with_meta['parent_id']}")
    
    if synth_with_meta.get('metadata'):
        meta = synth_with_meta['metadata']
        print(f"\nMetadata:")
        print(f"  Samples: {meta['num_samples']}")
        print(f"  Sample IDs: {meta['sample_ids']}")
        print(f"  Model homogeneous: {meta['is_homogeneous_model']}")
        print(f"  Models used: {meta['models_used']}")

# Get full provenance tree
if store.list_syntheses('principles_guide'):
    print("\n\nFull Provenance Tree")
    print("=" * 50)
    
    principles_id = store.list_syntheses('principles_guide')[0]
    provenance = store.get_synthesis_provenance(principles_id)
    
    print(f"\nPrinciples Guide: {provenance['synthesis']['synthesis_id']}")
    print(f"  Created: {provenance['synthesis']['created_at']}")
    print(f"  Model: {provenance['synthesis']['model']}")
    
    if provenance['parent']:
        parent = provenance['parent']
        print(f"\n  Parent (Cross-Text): {parent['synthesis']['synthesis_id']}")
        print(f"    Sample contributions: {len(parent['sample_contributions'])}")
        for sample_id, analyst in parent['sample_contributions'][:3]:
            print(f"      - {sample_id} / {analyst}")
        if len(parent['sample_contributions']) > 3:
            print(f"      ... and {len(parent['sample_contributions']) - 3} more")

## Exporting Syntheses to Filesystem

Export final syntheses to text files with YAML metadata headers for consumption by other tools or LLMs.

In [ ]:
# Create outputs directory
outputs_dir = Path("outputs")
outputs_dir.mkdir(exist_ok=True)

# Export cross-text synthesis
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
if cross_text_syntheses:
    for synth_id in cross_text_syntheses:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")

# Export principles guide
principles_guides = store.list_syntheses('principles_guide')
if principles_guides:
    for synth_id in principles_guides:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")
        
        # Also create a special "derived_style_instructions.txt" for style_evaluation.ipynb
        if synth_id == principles_guides[-1]:  # Use latest
            instructions_path = outputs_dir / "derived_style_instructions.txt"
            store.export_synthesis(synth_id, instructions_path, metadata_format='yaml')
            print(f"Exported for style evaluation: {instructions_path}")

print(f"\nAll syntheses exported to {outputs_dir.absolute()}")

## Utilities: Working with Stored Samples

Helper functions for browsing and managing stored results.

In [ ]:
# List all samples in the database
all_samples = store.list_samples()
print(f"Total samples: {len(all_samples)}")
print(f"Sample IDs: {all_samples}")

# Check completion status for each
print("\nCompletion status:")
for sid in all_samples:
    complete = store.is_complete(sid, ANALYSTS)
    status = "✓" if complete else "✗"
    print(f"  {status} {sid}")

# Close database connection when done
# store.close()